In [35]:
from pybtex.database.input import bibtex
import os

import pandas as pd
from nameparser import HumanName

In [36]:
df = pd.read_csv("my-talks.tsv", sep="\t", header=0,parse_dates=['date','date-start','date-end'])
display(df)
display(df.columns)

,title,key,type,conference,slides,poster,video,organization,date,date-start,...,location,url,abstract,author-01,author-02,author-03,author-04,author-05,Unnamed: 19,Unnamed: 20
0,GPU Accelerated Wildfire Smoke Simulations,mittag-2024,invited,Numerical Methods for PDEs and their Applications,mittag-2024,NaN,https://www.mittag-leffler.se/activities/numer...,Institut Mittlag-Leffler,2024-06-06,2024-06-03,...,"Djursholm, Sweden",https://www.mittag-leffler.se/activities/numer...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"Tree-based Adaptive Mesh Refinement, GPUs (and...",kaust-2024,invited,KAUST Turbulence Workshop,kaust-2024.pdf,NaN,NaN,KAUST,2024-02-27,2024-02-26,...,"Thuwal, Saudi Arabia",https://turbulenceworkshop.kaust.edu.sa/,We describe an approach to adaptive mesh refin...,Donna Calhoun,NaN,NaN,NaN,NaN,NaN,NaN
2,Coupling scientific software through the adapt...,cu-2024,invited,Applied Mathematics Seminar,cu-2024.pdf,NaN,NaN,"University of Colorado, Boulder",2024-02-09,NaT,...,"Boulder, CO (USA)",NaN,Coupling scientific research codes presents se...,Donna Calhoun,Carsten Burstedde,Scott Aiton,NaN,NaN,NaN,NaN
3,"ForestClaw : Parallel, adaptive mesh refinemen...",snubic-2023,invited,SNuBIC! Research Seminar,snubic-2023.pdf,NaN,NaN,"SNuBIC Research Group (Aachen, Bochum, Cologne...",2023-12-08,NaT,...,Virtual,NaN,Large scale simulations place extreme computat...,Donna Calhoun,Scott Aiton,Carsten Burstedde,NaN,NaN,NaN,NaN
4,"ForestClaw : Parallel, adaptive mesh refinemen...",siam-pnw-2023,invited-mini-symposium,SIAM Pacific Northwest Section Meeting,siam-pnw-2023.pdf,NaN,NaN,Society for Industrial and Applied Mathematics,2023-10-14,2023-10-13,...,"Bellingham, WA (USA)",https://sites.google.com/view/siampnw23/home,Large scale geophysical simulations place extr...,Donna Calhoun,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58,Numerical Solution of the Bloch-Torrey Equatio...,siam-cse-bt-2013,invited-mini-symposium,SIAM Conference on Computational Science and E...,siam-cse-bt-2013.pdf,NaN,NaN,Society for Industrial and Applied Mathematics,2013-02-25,2013-02-25,...,"Boston, MA",https://archive.siam.org/meetings/cse13/,We propose a numerical method to solve the Blo...,Donna Calhoun,Jing-Rebecca Li,NaN,NaN,NaN,NaN,NaN
59,A hybrid adaptive mesh framework for finite vo...,siam-geo-2013,mini-symposium-organizer,SIAM Conference on Mathematical and Computatio...,siam-geo-2013.pdf,NaN,NaN,Society for Industrial and Applied Mathematics,2013-06-20,2013-06-17,...,"Padua, Italy",https://archive.siam.org/meetings/gs13/,Skepticism persists as to whether the use of a...,Donna Calhoun,Carsten Burstedde,NaN,NaN,NaN,NaN,NaN
60,Solving the Riemann problem for linear and lin...,ini-riemann-2012,workshop,Multiscale Numerics for the Ocean and Atmosphere,ini-riemann-2012.pdf,NaN,NaN,"Isaac Newton Institute, Cambridge University, ...",2012-11-26,2012-08-23,...,"Cambridge, England",https://www.newton.ac.uk/event/amm/,Introduction to solving Riemann problems for l...,Donna Calhoun,NaN,NaN,NaN,NaN,NaN,NaN
61,"A logically Cartesian, adaptively refined two-...",warwick-2012,invited,Applied Math Seminar,warwick-2012.pdf,NaN,NaN,University of Warwick,2012-11-16,NaT,...,"Conventry, UK",NaN,"Recently, we developed a novel, logically Cart...",Donna Calhoun,Carsten Burstedde,NaN,NaN,NaN,NaN,NaN


Index(['title', 'key', 'type', 'conference', 'slides', 'poster', 'video',
       'organization', 'date', 'date-start', 'date-end', 'location', 'url',
       'abstract', 'author-01', 'author-02', 'author-03', 'author-04',
       'author-05', 'Unnamed: 19', 'Unnamed: 20'],
      dtype='object')

In [37]:
talk_type_list = df.type.unique()
print(talk_type_list)

['invited' 'invited-mini-symposium' 'colloquium' 'poster' 'keynote'
 'workshop' 'mini-symposium-organizer' 'tutorial']


In [38]:
# Create a map to get info that can be used in a citation.

talk_type_map = {'poster' : 'poster', \
                'colloquium' : 'colloquium', \
                'invited' : 'invited', \
                'keynote' : 'Keynote', \
                'invited-mini-symposium' : 'Mini-symposium speaker',
                'workshop' : 'Workshop lecture',
                'tutorial' : 'Tutorial',
                'mini-symposium-organizer' : "Mini-symposium organizer/speaker"}


In [39]:
html_escape_table = {
    "&": "&amp;",
    '"': "&quot;",
    "'": "&apos;"
    }

def html_escape(text):
    """Produce entities within text."""
    return "".join(html_escape_table.get(c,c) for c in text)

In [40]:
# df_journals = df[df['journal'].isnull() == False]
def single_quote(s):
    return "'" + s + "'"

def double_quote(s):
    return "\"" + s + "\""

def endl():
    return "\n"

def br():
    return "<br/>"

refereed = ['article', 'inproceedings']

for row, item in df.iterrows():    
    talk_type = item['type']
    talk_id = item['key']
    citekey = talk_id.replace(':','-')
    md_filename = citekey + ".md"
    html_filename = citekey + ".html"
    # pdf_filename = item['file']
    date = item['date']
    date_start = item['date-start']
    date_end = item['date-end']
    
    year = date.year
    print(year)
    
    ## YAML variables
    
    md = "---" + endl()
    # ------
    # title: 
    # ------
    md += "title: " + single_quote(item['title']) + endl()
    
    # -----------
    # collection: 
    # -----------
    md += "collection: talks" + endl()

    
    # -----------
    # talk_type:
    # -----------
    
    talk_type_mode =  talk_type_map[talk_type]
    md += "type: " + talk_type_mode.capitalize() + endl()
    md += "talk_type: " + talk_type_mode.capitalize() + endl()
    
    # ----------
    # permalink: 
    # ----------
    # md += "permalink: /talks/" + html_filename + endl()
    md += 'permalink: /talks/' + html_filename + endl()
    
    # ---------
    # abstract:
    # ---------
    if not pd.isna(item['abstract']):
        md += "abstract: " + single_quote(html_escape(item['abstract'])) + endl()

    # ---------
    # slides:
    # ---------
    if not pd.isna(item['slides']):
        md += 'slides: ' + single_quote(item['slides']) + endl()
        
    if not pd.isna(item['poster']):
        md += 'poster: ' + single_quote(item['poster']) + endl()

    # ---------
    # video:
    # ---------
    if not pd.isna(item['video']):
        md += 'video: ' + single_quote(item['video']) + endl()
                
    # -----
    # date: 
    # -----
    
    md += "date: " + date.strftime('%Y-%m-%d') + endl()
    
    date_talk = date.strftime("%b %d, %Y")
    md += "date_talk: " + date_talk + endl()

    date_start_end = ""
    if not (pd.isna(item['date-start']) or pd.isna(item['date-end'])):
        date_start_end = "{:s} - {:s}, {:s}".format(\
                                date_start.strftime("%b %d"), \
                                date_end.strftime("%b %d"), date_end.strftime("%Y"))
        md += "date_conference: {:s}".format(single_quote(date_start_end)) + endl()
    
    
    # ------
    # venue: 
    # ------
    md += "venue: " + single_quote(item['conference']) + endl()

    # -------------
    # organization: 
    # -------------
    md += "organization: " + single_quote(item['organization']) + endl()
    
    # ---------------
    # conference_url: 
    # ---------------
    
    if not pd.isna(item['url']):
        url = item['url']
        md += "conference_url: " + single_quote(html_escape(url)) + endl()

    # ---------
    # location:
    # ---------
    md += 'location: {:s}'.format(item['location']) + endl()
        
    # ----------------------------------------
    # Authors
    # ----------------------------------------
    
    author_list = ''
    for i in range(1,6):        
        author = item['author-{:02}'.format(i)]
        if not pd.isna(author):
            a = HumanName(author)
            author_list += a.first + ' ' + a.last + ', '
            
    if author_list != '':
        author_list = author_list[:-2] + endl();
        md += 'authors: ' + author_list
            
                
    md += "---" + endl()
    
    
    print("Processed file : {:s}".format(citekey))
    print(talk_type)
    print(date_talk)
    print(author_list)
    print(talk_type_mode)
    print(date_start_end)
    print("")

    
    # ----------------------------------------
    # Markdown description for individual page
    # ----------------------------------------
    
    # Put miscellaneous content here
    
    # md += ...
    
    # ----------------------------------------
    # Write out file name
    # ----------------------------------------

    md_filename = os.path.basename(md_filename)
       
    with open("../_talks/" + md_filename, 'w') as f:
        f.write(md)    
        
print("Success!")

2024
Processed file : mittag-2024
invited
Jun 06, 2024

invited
Jun 03 - Jun 07, 2024

2024
Processed file : kaust-2024
invited
Feb 27, 2024
Donna Calhoun

invited
Feb 26 - Feb 28, 2024

2024
Processed file : cu-2024
invited
Feb 09, 2024
Donna Calhoun, Carsten Burstedde, Scott Aiton

invited


2023
Processed file : snubic-2023
invited
Dec 08, 2023
Donna Calhoun, Scott Aiton, Carsten Burstedde

invited


2023
Processed file : siam-pnw-2023
invited-mini-symposium
Oct 14, 2023
Donna Calhoun

Mini-symposium speaker
Oct 13 - Oct 15, 2023

2023
Processed file : siam-cse-2023
invited-mini-symposium
Mar 02, 2023
Donna Calhoun, Scott Aiton, Carsten Burstedde

Mini-symposium speaker
Feb 27 - Mar 03, 2023

2022
Processed file : bsu-2022
colloquium
Sep 13, 2022

colloquium


2022
Processed file : siam-pnw-2022
invited-mini-symposium
May 22, 2022
Donna Calhoun, Jodi Mead, Patricia Azike, Sandra Babyale

Mini-symposium speaker
May 20 - May 22, 2022

2022
Processed file : icerm-2022
invited
Jan 12, 2

In [41]:
print("Done")

Done
